In [1]:
!pip install ipwhois

In [2]:
import socket
import requests
import subprocess
import platform

from ipwhois import IPWhois
import matplotlib.pyplot as plt
import os

def get_ip_address():
    hostname = socket.gethostname()
    ip_address = socket.gethostbyname(hostname)
    return ip_address

def check_https(url):
    try:
        response = requests.get(url)
        return response.url.startswith("https://")
    except requests.exceptions.RequestException:
        return False

def traceroute(host):
    os_name = platform.system()
    if os_name == "Windows":
        command = ['tracert', host]
    else:
        command = ['traceroute', host]
    
    try:
        result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if result.returncode == 0:
            return result.stdout.decode('utf-8').splitlines()
        else:
            return []
    except Exception:
        return []

def geolocate_ip(ip):
    try:
        obj = IPWhois(ip)
        res = obj.lookup_rdap()
        return res['asn_country_code'], res['asn_description']
    except Exception:
        return "Unknown", "Unknown"

def plot_latency(hops, latencies):
    plt.figure(figsize=(10, 5))
    plt.plot(hops, latencies, marker='o')
    plt.title('Traceroute Latency')
    plt.xlabel('Hop')
    plt.ylabel('Latency (ms)')
    plt.grid(True)
    plt.savefig('latency_plot.png')  # Save the plot to a file
    plt.show()  # Ensure the plot is displayed

def main():
    # Check IP Address
    ip_address = get_ip_address()
    print(f"Your IP address is: {ip_address}")

    # Check HTTPS for a common site
    url = "https://www.google.com"
    is_secure = check_https(url)
    print(f"The connection to {url} is secure (HTTPS)." if is_secure else f"The connection to {url} is not secure (HTTP).")

    # Perform a traceroute
    traceroute_output = traceroute("www.google.com")

    # Analyze traceroute output
    hops = []
    latencies = []
    for line in traceroute_output:
        if line.startswith(" "):
            parts = line.split()
            if len(parts) >= 9:
                hop = parts[1]
                latency = parts[4]
                ip = parts[3]
                try:
                    latencies.append(float(latency.replace('ms', '')))
                    hops.append(hop)
                    country, description = geolocate_ip(ip)
                    print(f"Hop: {hop}, IP: {ip}, Latency: {latency}, Location: {country}, Description: {description}")
                except ValueError:
                    print(f"Skipping hop with invalid latency: {line}")

    # Plot latency
    if hops and latencies:
        plot_latency(hops, latencies)

# Run the main function
main()


ModuleNotFoundError: No module named 'ipwhois'